# Ejercicio 3 (análisis de sentimiento): (3 puntos)
Se pide desarrollar un notebook de Jupyter, denominado “sentiment.ipynb”, en el que se utilice como fuente de datos Kafka, y en concreto el topic kafkaTwitter. Para cada nuevo tweet que llegue al sistema, se analizará su polaridad dividiendo el tweet por palabras y realizando la suma de la polaridad de todas las palabras que aparezcan en los ficheros “positive_lex.txt” y “negative_lex.txt”. Las palabras del lexicon negativo tienen un signo menos delante, por lo que restarán a la puntuación final del tweet. Si ninguna de las palabras del tweet está almacenada en los ficheros proporcionados (o si las puntuaciones de las palabras positivas y negativas se anulan), la polaridad del tweet será 0.

La salida del sistema ha de imprimirse cada segundo, y consistirá en una línea por cada tweet recibido, que contenga el texto del propio tweet, a continuación la puntuacin de su polaridad, y la palabra “NEUTRO” si la puntuación es igual 0, “POSITIVO” si es mayor que 0, y “NEGATIVO” si es menor que 0.


In [1]:
import findspark
findspark.init('/opt/spark')

In [2]:
import os
os.environ['JAVA_HOME'] = '/Library/Java/JavaVirtualMachines/jdk1.8.0_171.jdk/Contents/Home/jre'
os.environ['PYSPARK_SUBMIT_ARGS'] = '--master local[*] --packages org.apache.spark:spark-streaming-kafka-0-8_2.11:2.4.4 pyspark-shell'

In [3]:
import pyspark
import pyspark.streaming
from pyspark import SparkConf, SparkContext
from pyspark.sql import SparkSession
from pyspark.streaming import StreamingContext
from pyspark.streaming.kafka import KafkaUtils

In [4]:
spark = SparkSession.builder.appName("hashtags").master("local[*]").getOrCreate()

# 1s de batchDuration
ssc = StreamingContext(spark.sparkContext, 1)

ssc.checkpoint("checkpoint")

## Empezamos por cargar los datos de positive_lex.txt y negative_lex.txt a un DataFrame

In [5]:
# Parametrizar con ruta donde están los ficheros.
neg_file_path = "DocumentosTP3/negative_lex.txt"
raw_neg_lex = spark.read.option("header", "false").csv(neg_file_path)

In [6]:
raw_neg_lex.show(5)

+------------------+
|               _c0|
+------------------+
|     abatir -0.554|
| abochornado -0.25|
|   abochornar -0.5|
|abominación -0.425|
|  aborrecer -0.625|
+------------------+
only showing top 5 rows



In [7]:
# Parametrizar con ruta donde están los ficheros.
pos_file_path = "DocumentosTP3/positive_lex.txt"
raw_pos_lex = spark.read.option("header", "false").csv(pos_file_path)

In [8]:
raw_pos_lex.show(5)

+---------------+
|            _c0|
+---------------+
| acertado 0.708|
|admirable 0.906|
|admiración 0.45|
|   admirar 0.75|
| afectivo 0.375|
+---------------+
only showing top 5 rows



Separamos la información en 2 columnas para que sea mas facil de procesar.

In [9]:
from pyspark.sql import Column, DataFrame, functions

In [10]:
neg_col = functions.split(raw_neg_lex._c0," ")
neg_lex = raw_neg_lex.select(neg_col[0].alias("word"), neg_col[1].alias("puntuacion"))

In [11]:
neg_lex.show(5)

+-----------+----------+
|       word|puntuacion|
+-----------+----------+
|     abatir|    -0.554|
|abochornado|     -0.25|
| abochornar|      -0.5|
|abominación|    -0.425|
|  aborrecer|    -0.625|
+-----------+----------+
only showing top 5 rows



In [12]:
pos_col = functions.split(raw_pos_lex._c0," ")
pos_lex = raw_pos_lex.select(pos_col[0].alias("word"), pos_col[1].alias("puntuacion"))

In [13]:
pos_lex.show(5)

+----------+----------+
|      word|puntuacion|
+----------+----------+
|  acertado|     0.708|
| admirable|     0.906|
|admiración|      0.45|
|   admirar|      0.75|
|  afectivo|     0.375|
+----------+----------+
only showing top 5 rows



## Vamos a cruzar las palabras de los tweets con las tablas.

Primero debemos separar el DStream en los distintos tweets y quedarnos solo con el mensaje.

In [14]:
# DStream con el batch de 5s de Kafka.
kafkaStream = KafkaUtils.createStream(ssc,zkQuorum='localhost:2181',groupId="group_1" , topics={'kafkaTwitter':2})

In [15]:
#tweet_stream = kafkaStream.map(lambda tweet: tweet[1],)

Calculamos la puntuación por palabra.

In [16]:
pos_dict = {}
for row in pos_lex.collect():
      pos_dict[row.word] = float(row.puntuacion)
pos_dict

{'acertado': 0.708,
 'admirable': 0.906,
 'admiración': 0.45,
 'admirar': 0.75,
 'afectivo': 0.375,
 'afecto': 0.321,
 'afectuoso': 0.563,
 'afición': 0.5,
 'afortunado': 0.813,
 'agradable': 0.75,
 'agradecido': 0.5,
 'alegrar': 0.458,
 'alegre': 0.661,
 'alentador': 0.25,
 'alentar': 0.275,
 'amable': 0.688,
 'amar': 1.0,
 'amistoso': 0.708,
 'amor': 0.278,
 'animado': 0.494,
 'animar': 0.289,
 'apacible': 0.313,
 'apasionado': 0.469,
 'apego': 0.375,
 'aprecio': 0.344,
 'aprobar': 0.375,
 'ardiente': 0.313,
 'atractivo': 0.5,
 'beneficio': 0.396,
 'benevolencia': 0.438,
 'benevolente': 0.313,
 'bien': 0.792,
 'bienestar': 0.531,
 'bondad': 0.25,
 'bonito': 0.75,
 'buen': 0.734,
 'bueno': 0.5,
 'cariñoso': 0.438,
 'cautivado': 0.438,
 'cautivador': 0.594,
 'cercano': 0.354,
 'comodidad': 0.406,
 'compañerismo': 0.25,
 'competente': 0.589,
 'complacencia': 0.688,
 'complacer': 0.475,
 'complaciente': 0.643,
 'confianza': 0.375,
 'confortar': 0.375,
 'consolar': 0.375,
 'contentarse': 

In [17]:
pos_dict['acertado']

0.708

In [18]:
neg_dict = {}
for row in neg_lex.collect():
      neg_dict[row.word] = float(row.puntuacion)
neg_dict

{'abatir': -0.554,
 'abochornado': -0.25,
 'abochornar': -0.5,
 'abominación': -0.425,
 'aborrecer': -0.625,
 'aborrecimiento': -0.425,
 'acongojarse': -0.687,
 'acosar': -0.304,
 'aflicción': -0.606,
 'afligido': -0.396,
 'afligir': -0.675,
 'afligirse': -0.687,
 'agravar': -0.281,
 'agresividad': -0.35,
 'alarmar': -0.25,
 'alboroto': -0.25,
 'amargura': -0.5,
 'angustia': -0.589,
 'angustiado': -0.65,
 'antipatía': -0.542,
 'antipático': -0.333,
 'apesadumbrado': -0.5,
 'apocamiento': -0.406,
 'apurarse': -0.35,
 'arrepentido': -0.5,
 'arriesgado': -0.25,
 'asco': -0.357,
 'asquear': -0.437,
 'asqueroso': -0.575,
 'atormentar': -0.391,
 'atroz': -0.672,
 'avergonzado': -0.25,
 'avergonzar': -0.375,
 'aversión': -0.359,
 'cabreado': -0.75,
 'celoso': -0.25,
 'colérico': -0.458,
 'confundir': -0.333,
 'congoja': -0.75,
 'conmoción': -0.354,
 'consternado': -0.375,
 'contrariedad': -0.656,
 'contrito': -0.5,
 'crueldad': -0.525,
 'culpabilidad': -1.0,
 'culpable': -0.312,
 'dañino': -0

In [19]:
neg_dict['abatir']

-0.554

In [20]:
# Función para calcular la puntuación por palabra y por fichero base.
def getWordScore(word, base_words):
    puntuacion = 0.0
    if type(word) is str :
        if word in base_words :
            puntuacion = base_words[word]
    return puntuacion

In [21]:
# Función para calcular puntuación por frase.
def getSentenceScore(sentence):
    puntuacion = 0.0
    if type(sentence) is str :
        for word in sentence.split():
            word_lower = word.lower()
            puntuacion += getWordScore(word_lower, pos_dict)
            puntuacion += getWordScore(word_lower, neg_dict)
    return puntuacion

In [22]:
getWordScore('abatir', neg_dict)

-0.554

In [23]:
getWordScore('afición', pos_dict)

0.5

In [24]:
getSentenceScore('aBaTir Gabilondo: "Sí. Hay que devolver la tarjeta sanitaria a los inmigrantes" http://t.co/84g2BqqQjU')

-0.554

### Creamos una función que devuelva si POSITIVO, NEUTRO o NEGATIVO

In [25]:
def score2Word(score):
    if score < 0.0: word = 'NEGATIVO'
    elif score > 0.: word = 'POSITIVO'
    else: word = 'NEUTRO'
    return word

In [26]:
score2Word(0.0)

'NEUTRO'

In [27]:
score2Word(0.1)

'POSITIVO'

In [28]:
score2Word(-0.1)

'NEGATIVO'

### Por último ejecutamos las funciones por cada mensaje recibido en el topic de kafka e imprimimos

In [29]:
tweet_score = kafkaStream.map( lambda tweet: (tweet[1], getSentenceScore(tweet[1]))).map(lambda t_p: (t_p[0], t_p[1] , score2Word(t_p[1])))

In [30]:
tweet_score.pprint()

In [31]:
ssc.start()

-------------------------------------------
Time: 2020-01-12 21:28:37
-------------------------------------------

-------------------------------------------
Time: 2020-01-12 21:28:38
-------------------------------------------
('@Olicaminos en serio me lo contas geniaaa!! :D', 0.0, 'NEUTRO')
('"El acuerdo en Cataluña es que aparezca el nombre de Podemos, con un guión y el resto de apoyos" @Pablo_Iglesias_ #PabloIglesiasM4', -0.375, 'NEGATIVO')
('@maariisol dejaste de estar en la Friendzone :D', 0.0, 'NEUTRO')
('Demostrame que, tu amor vale la pena,asi yo me la juego y empezamos hacer la nuestra.🎶', 0.278, 'POSITIVO')
('Resumen de Twitter hoy:.iOS 7.GTA V .iOS 7.iOS 7.GTA V  .iOS 7.GTA V ..Y algún tipo que tuiteo ¡Feliz año 2011! desde Internet Explorer.', 0.0, 'NEUTRO')
('"Nunca había visto tanta gente en este colegio como hoy", dice un apoderado en #Madrid Y así en toda España. #PSOE24M http://t.co/OAzsk74sHH', -0.375, 'NEGATIVO')
('@apotema69 jo, mierda jajajaja 😂😂😂 Bon dia guapo :